# Lab 2-1. Logistic Regression

*   Regression: input data $x$를 output data $y$로 매핑하는 모델 $f$를 찾는것.

*   Logistic Regression: 모델 $f$가 linear model의 output을 logistc function을 이용해서 0~1사이의 확률값을 예측함, 즉 $$f(\textbf{x})=\sigma(\textbf{w}^T\textbf{x}+b)=\dfrac{1}{1+e^{-(\textbf{w}^T\textbf{x}+b)}}$$

*   Loss function
$$\text{Loss}(\textbf{w})=-\dfrac{1}{N}\sum y\text{log}(f(\textbf{x}))+(1-y)\text{log}(1-f(\textbf{x})) $$   

*    Gradient Descent
$$\textbf{w}=\textbf{w}-\alpha \dfrac{\partial}{\partial\textbf{w}}\text{Loss}(\textbf{w})$$

#### Import torch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Logistic Regression with Toy Training Data

In [2]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [3]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [4]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


#### Compute the Hypothesis
$$f(\textbf{x})=\sigma(\textbf{w}^T\textbf{x}+b)=\dfrac{1}{1+e^{-(\textbf{w}^T\textbf{x}+b)}}$$


In [5]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [6]:
W = torch.zeros((2, 1), requires_grad=True) # requires_grad : 경사하강법 사용시 설정
b = torch.zeros(1, requires_grad=True)

In [7]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [8]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


`torch.sigmoid()` 함수를 쓸 수도 있음


In [9]:
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals:  tensor([0.7311])


In [10]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [11]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


#### Computing the Loss Function (Low-level)

$$\text{Loss}(\textbf{w})=-\dfrac{1}{N}\sum y\text{log}(f(\textbf{x}))+(1-y)\text{log}(1-f(\textbf{x})) $$   

 `hypothesis` and `y_train` 사이의 차이를 측정하고 싶음.

In [12]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


하나의 데이터에 대해서 다음과 같이 계산 가능

In [13]:
-(y_train[0] * torch.log(hypothesis[0]) + 
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward0>)

전체 데이터에 대해서는 다음과 같이 간단하게 계산 가능

In [14]:
losses = -(y_train * torch.log(hypothesis) + 
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


`.mean()` 을 사용하여 평균을 취함

In [15]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


#### Computing the Cost Function with `F.binary_cross_entropy`

실제로는 binary classification은 자주 쓰이기 때문에 pytorch에 `F.binary_cross_entropy` 라는 함수가 구현돼있음

(지난번 실습의 `torch.nn.MSELoss`처럼)

In [16]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

#### Training with Low-level Binary Cross Entropy Loss

In [17]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [18]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


#### Training with `F.binary_cross_entropy`

In [19]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Logistic Regression with Real Data

### Load File from Google Drive

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
diabetes = pd.read_csv('/content/drive/MyDrive/프로그래밍_교육/쏘카 ai 엔지니어 육성 부트캠프 2기/강의/2주차/Chapter 2_실습파일/diabetes.csv')
print(diabetes.columns)
diabetes.head()

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [24]:
type(diabetes)

pandas.core.frame.DataFrame

In [23]:
print("dimension of diabetes data: {}".format(diabetes.shape))

dimension of diabetes data: (768, 9)


### Split and prepare the dataset

In [25]:
train = diabetes[:650]
test = diabetes[650:]

In [26]:
print(train.groupby('Outcome').size())

Outcome
0    427
1    223
dtype: int64


imbalanced dataset임을 확인 가능

In [27]:
x_train = np.asarray(train.drop('Outcome',1))
y_train = np.asarray(train['Outcome'])
x_test = np.asarray(test.drop('Outcome',1))
y_test = np.asarray(test['Outcome'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


input을 normalize 해준다.

In [28]:
means = np.mean(x_train, axis=0)
stds = np.std(x_train, axis=0)
x_train = (x_train - means)/stds
x_test = (x_test - means)/stds

In [29]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train).unsqueeze(-1) # .unsqueeze(-1) 차원을 하나 늘려준다
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test).unsqueeze(-1)

In [30]:
y_train.shape

torch.Size([650, 1])

### Training with F.binary_cross_entropy

In [31]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 f(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 0.693147
Epoch   10/100 Cost: 0.477473
Epoch   20/100 Cost: 0.468751
Epoch   30/100 Cost: 0.467308
Epoch   40/100 Cost: 0.467003
Epoch   50/100 Cost: 0.466932
Epoch   60/100 Cost: 0.466914
Epoch   70/100 Cost: 0.466910
Epoch   80/100 Cost: 0.466909
Epoch   90/100 Cost: 0.466908
Epoch  100/100 Cost: 0.466908


### Test Accuracy

In [32]:
W, b

(tensor([[ 0.4520],
         [ 1.0729],
         [-0.2492],
         [-0.0065],
         [-0.1097],
         [ 0.7790],
         [ 0.3553],
         [ 0.1201]], requires_grad=True),
 tensor([-0.9190], requires_grad=True))

In [33]:
hypothesis = torch.sigmoid(x_test.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])
correct_prediction = prediction.float() == y_test
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the test set.'.format(accuracy * 100))

The model has an accuracy of 78.81% for the test set.


## (Optional) Balancing the training set

In [34]:
x_train = np.asarray(train.drop('Outcome',1))
y_train = np.asarray(train['Outcome'])
x_test = np.asarray(test.drop('Outcome',1))
y_test = np.asarray(test['Outcome'])
means = np.mean(x_train, axis=0)
stds = np.std(x_train, axis=0)
x_train = (x_train - means)/stds
x_test = (x_test - means)/stds

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
x_train_pos = x_train[y_train == 0]
x_train_neg = x_train[y_train == 1]
y_train_pos = y_train[y_train == 0]
y_train_neg = y_train[y_train == 1]

In [36]:
x_train_pos.shape, x_train_neg.shape

((427, 8), (223, 8))

In [37]:
ids = np.arange(len(x_train_neg))
choices = np.random.choice(ids, len(x_train_pos))
x_train_neg = x_train_neg[choices]
y_train_neg = y_train_neg[choices]

In [38]:
x_train_pos.shape, x_train_neg.shape

((427, 8), (427, 8))

In [39]:
x_train = np.concatenate([x_train_pos, x_train_neg], axis=0)
y_train = np.concatenate([y_train_pos, y_train_neg], axis=0)

order = np.arange(len(x_train))
np.random.shuffle(order)
x_train = x_train[order]
y_train = y_train[order]

In [40]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train).unsqueeze(-1)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test).unsqueeze(-1)

### Training with F.binary_cross_entropy

In [41]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 f(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 0.693147
Epoch   10/100 Cost: 0.508700
Epoch   20/100 Cost: 0.502603
Epoch   30/100 Cost: 0.501680
Epoch   40/100 Cost: 0.501503
Epoch   50/100 Cost: 0.501465
Epoch   60/100 Cost: 0.501456
Epoch   70/100 Cost: 0.501454
Epoch   80/100 Cost: 0.501454
Epoch   90/100 Cost: 0.501454
Epoch  100/100 Cost: 0.501454


### Test Accuracy

In [42]:
hypothesis = torch.sigmoid(x_test.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])
correct_prediction = prediction.float() == y_test
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuracy of 74.58% for the training set.
